# DGI Portfolio Builder

This notebook demonstrates the full pipeline of the DGI toolkit: loading fundamentals, screening, scoring, building a portfolio, plotting weights, and printing summary statistics.

In [ ]:
import sys
print(f'Python version: {sys.version}')
import os
print('Current working directory:', os.getcwd())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from dgi.screener import Screener
from dgi.validation import DgiRowValidator
from dgi.repositories.csv import CsvCompanyDataRepository
from dgi.scoring import DefaultScoring
from dgi.filtering import DefaultFilter
from dgi.portfolio import build, summary_stats

## Load Raw Fundamentals Data

In [ ]:
csv_path = '../data/fundamentals_small.csv'
repo = CsvCompanyDataRepository(csv_path, DgiRowValidator())
screener = Screener(repo, scoring_strategy=DefaultScoring(), filter_strategy=DefaultFilter())
df = screener.load_universe()
df.head()

## Apply Filters

In [ ]:
filtered = screener.apply_filters(df, min_yield=0.5, max_payout=60, min_cagr=5.0)
filtered.head()

## Score and Rank

In [ ]:
scored = screener.add_scores(filtered)
scored = scored.sort_values('score', ascending=False)
scored.head()

## Build Portfolio (Top N, Equal Weight)

In [ ]:
top_n = 5
portfolio = build(scored, top_n=top_n, weighting='equal')
portfolio.head()

## Plot Portfolio Weights

In [ ]:
plt.figure(figsize=(8, 4))
plt.bar(portfolio['ticker'], portfolio['weight'], color='skyblue')
plt.xlabel('Ticker')
plt.ylabel('Weight')
plt.title('Portfolio Weights')
plt.show()

## Portfolio Summary Statistics

In [ ]:
# Merge to get stats columns
merged = portfolio.merge(scored, left_on='ticker', right_on='symbol', how='left')
stats = summary_stats(merged)
print('Portfolio Yield:', stats['yield'])
print('Median CAGR:', stats['median_cagr'])
print('Mean Payout:', stats['mean_payout'])